# Data assimilation applied to a shallow water model: practical sessions

##### Alban Farchi, CEREA, [alban.farchi@enpc.fr](mailto:alban.farchi@enpc.fr)

During these sessions, you will apply two classical data assimilation methods to a shallow water model. The objective for you is to better understand these methods, figure out their practical implementations and identify their key parameters.

## I. The shallow water model

A shallow water model is well adapted to flows below a free surface when the depth is much smaller than the horizontal dimensions. It is commonly used to represent a lake or river. Its equations describe the time evolution of the water height $h(x)$ and the horizontal velocity $u(x)$ in a fixed-length domain.

A simplified unidimensional version reads
$$
    \frac{\partial h}{\partial t} + \frac{\partial(hu)}{\partial x} = 0,\\
    \frac{\partial(hu)}{\partial t} + \frac{\partial(hu^2)}{\partial x} + gh\frac{\partial h}{\partial x} = 0.
$$

Several boundary conditions may be defined. In our case, we rely on the following conditions:
1. on the left, a constant inflow $Q=hu$;
2. on the right, a homogeneous Neumann condition for $h$ and $u$, where fluxes are consequently determined by the state of the system along the boundary.

The equations are discretized and numerically solved in the custom `shallow_water_model` module.

## II. The truth simulation

In this series of experiments, we will use twin simulations.
1. We run a reference simulation. The result is considered to be the **true situation** and is called the **truth**.
2. From the truth we extract **synthetic observations**.
3. Using the observations only, we try to reconstruct the truth using a dedicated **data assimilation** algorithm.

Let us start with the truth. The simulation domain is discretised using `Nx=101` grid points. At the initial time, the horizontal velocity $u(x)$ is null, and the water height $h(x)$ is a crenel: $h(x)=1$ everywhere but in the center of the domain, where $h(x)=1.05$. The simulation is run for `Nt=500` time iterations. The other model parameters are `dx=1` (the horizontal step), `dt=0.03` (the time step), `Q=0.1` (the constant inflow on the left), and `g=9.81` (the acceleration due to gravity).

### Questions

1. What is the dimension of the state vector of the numerical model?
2. Run the truth simulation and explain the dynamics of the truth.

### Tips

- If the animations are too long to produce, reduce the dpi in the first cell (use typically 100) or increase `freq` to `15` or `20` in the last two cells.

In [ ]:
# import standard modules
import numpy as np
import scipy
from matplotlib import pyplot as plt
from matplotlib import animation
import seaborn as sns
from tqdm.auto import trange
from IPython.display import HTML

# for plot customisation
sns.set_context('notebook')
sns.set_style('darkgrid')
plt.rc('axes', linewidth=1)
plt.rc('axes', edgecolor='k')
plt.rc('figure', dpi=100)
palette = sns.color_palette('deep')

# import custom shallow water model package
import shallow_water_model as swm

In [ ]:
# list of parameters
Nx = 101
dx = 1
dt = 0.03
Q = 0.1
g = 9.81
Nt = 500
h_anom = 1.05

In [ ]:
# create the model
sw_model = swm.ShallowWaterModel(Nx, dx, dt, Q, g)

# create a driver
def forecast_driver(model, state, Nt):
    """Run a simulation of Nt time steps of the given model starting from state.
    
    Return the history of h and u.
    """
    # allocate memory
    hist = dict(Nt=Nt,
                h=np.empty((Nt+1, model.Nx)),
                u=np.empty((Nt+1, model.Nx)))
    
    # initialisation
    hist['h'][0] = state.h
    hist['u'][0] = state.u
    
    # run the Nt time steps
    for t in trange(Nt, desc='running forward model'):
        model.forward(state)
        hist['h'][t+1] = state.h
        hist['u'][t+1] = state.u
        
    # return history
    return hist

In [ ]:
# initialise and run the truth simulation
state = sw_model.new_state_crenel(h_anom)
hist_truth = forecast_driver(sw_model, state, Nt)

In [ ]:
# make a fancy animation for water height
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h'])],
                                  'Truth simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for horizontal velocity
anim = swm.make_fancy_animation_u([('Truth', hist_truth['u'])],
                                  'Truth simulation',
                                  freq=10)
HTML(anim.to_jshtml())

## III. The observations

At each time step, `Ny=3` observations are available: the water height values at $x=79$, $x=80$, and $x=81$. We assume that there is no observation error.

### Questions

1. Implement the `apply_observation_operator` function. Compute the observations and explain their time evolution.

### Tips

- Use `v[i]` to access the $i$-th element of vector $\mathbf{v}$.
- Use `v[i:j+1]` to select the subvector $\mathbf{v}_{i:j}\triangleq(v_{i}, \ldots, v_{j})$.

In [ ]:
# list of parameters
Ny = 3

In [ ]:
# observation function
def apply_observation_operator(h):
    """Apply the observation operator to the vector h.
    
    Arguments
    ---------
    h : numpy array of size Nx
        The values of $h(x)$ in the domain.
        
    Returns
    -------
    y : numpy array of size Ny
        The observations.
    """
    # TODO: implement it! 
    return h[79:82]

# create a driver
def observation_driver(hist):
    """Extract observations from a simulation.
    
    Use the `apply_observation_operator` function to compute
    the observations.
    
    Return the updated history.
    """
    # extract Nt
    Nt = hist['Nt']
    
    # allocate memory
    hist['y'] = np.empty((Nt+1, Ny))
    
    # run the Nt+1 observation steps
    for t in trange(Nt+1, desc='running observation'):
        hist['y'][t] = apply_observation_operator(hist['h'][t])
        
    # return history
    return hist

In [ ]:
# run observation driver to compute the observations
hist_truth = observation_driver(hist_truth)

In [ ]:
# plot the time series of observations
swm.plot_time_series_h([('$h(x=79)$', hist_truth['y'][:, 0]),
                        ('$h(x=80)$', hist_truth['y'][:, 1]),
                        ('$h(x=81)$', hist_truth['y'][:, 2])],
                       'Observations')
plt.show()

## IV. Simulation without assimilation

We first try a simulation without data assimilation. For this **perturbed simulation**, we use a different initial condition: the water height is $h(x)=1$ everywhere.

### Questions

1. Run the perturbed simulation and explain the time evolution of the error after the initial misfit.

In [ ]:
# list of parameters
pert_h_anom = 1

In [ ]:
# initialise and run the perturbed simulation
state = sw_model.new_state_crenel(pert_h_anom)
hist_pert = forecast_driver(sw_model, state, Nt)

In [ ]:
# make a fancy animation for water height
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h']),
                                   ('Pert.', hist_pert['h'])],
                                  'Truth vs Pert. simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# auxiliary function to compute the MAE
def compute_mae(h1, h2):
    """Compute the time series of MAE between h1 and h2."""
    return abs(h1-h2).mean(axis=1)

# auxiliary function to compute the RMSE
def compute_rmse(h1, h2):
    """Compute the time series of RMSE between h1 and h2."""
    return np.sqrt(((h1-h2)**2).mean(axis=1))

In [ ]:
# compute MAE/RMSE for the perturbed simulation
hist_pert['mae'] = compute_mae(hist_truth['h'], hist_pert['h'])
hist_pert['rmse'] = compute_rmse(hist_truth['h'], hist_pert['h'])

In [ ]:
# plot the time series of MAE
swm.plot_time_series_mae([('Pert. sim.', hist_pert['mae'])],
                         title='Error in water height')
plt.show()

In [ ]:
# plot the time series of RMSE
swm.plot_time_series_rmse([('Pert. sim.', hist_pert['rmse'])],
                         title='Error in water height')
plt.show()

## V. Simulation with optimal interpolation

We now use the BLUE formula to assimilate the observations. For simplicity, the BLUE formula is applied only to $h(x)$. As a consequence, $u(x)$ is not updated during the analysis. Furthermore, we assume that both background and observation error covariance matrices $\mathbf{B}$ and $\mathbf{R}$ are the identity matrix.

The **analysis** (the product of the data assimilation process) is compared to the truth using both the mean absolute error (MAE) and the root mean squared error (RMSE).

### Questions

1. Write down the BLUE formula. Give the size (or shape) of each term.
2. Implement the `compute_analysis_blue` function. Run the assimilation. At which time iteration does the assimilation start influencing the simulated height? Why precisely at that time?
3. At the three observed locations, where does the analysis stand, compared to the forecast and the observations? Why precisely at that place? In case you would like to move the analysis closer to the observations (at the observed locations), how would you proceed?
4. Comment on the time evolution of the forecast error.
5. Why is the analysis error (as measured by the MAE) at late iterations larger with data assimilation than without? How would you solve this issue (still using optimal interpolation)?

### Tips

- Use `M@P` to compute the matrix product $\mathbf{MP}$.
- Use `M@v` to compute the matrix vector product $\mathbf{Mv}$.
- Use `M.T` to compute the transpose of matrix $\mathbf{M}$.
- Use `np.linalg.inv(M)` to compute the inverse of matrix $\mathbf{M}$.


In [ ]:
# list of parameters
blue_h_anom = 1

In [ ]:
# BLUE analysis
def compute_analysis_blue(hb, B, y, R, H):
    """Compute the BLUE analysis.
    
    Arguments
    ---------
    hb : numpy array of size Nx
        The prior (or forecast) values of $h(x)$ in the domain.
    B : numpy array of shape Nx * Nx
        The background error covariance matrix.
    y : numpy array of size Ny
        The observations.
    R : numpy array of shape Ny * Ny
        The observation error covariance matrix.
    H : numpy array of shape Ny * Nx
        The observation operator.
        
    Returns
    -------
    ha : numpy array of size Nx
        The posterior (or analysis) values of $h(x)$ in the domain.
    """
    # TODO: implement it!
    K = B@H.T@np.linalg.inv(R+H@B@H.T)
    return hb + K@(y-H@hb)

# create a driver
def blue_driver(model, state, hist_t):
    """Run a BLUE simulation of the given model starting from the given state.
    
    At each time step, an analysis for h is performed using `compute_analysis_blue`.
    
    Returns the history.
    """
    # extract Nt
    Nt = hist_t['Nt']
    
    # background error covariance matrix
    B = np.identity(model.Nx)

    # observation error covariance matrix
    R = np.identity(Ny)

    # observation operator
    H = np.zeros((Ny, model.Nx))
    H[:, 79:82] = np.identity(3)
    
    # allocate memory
    hist = dict(Nt=Nt,
                u=np.empty((Nt+1, model.Nx)),
                hf=np.empty((Nt+1, model.Nx)),
                ha=np.empty((Nt+1, model.Nx)))
    
    # initialisation
    hist['u'][0] = state.u
    hist['hf'][0] = state.h

    # run first analysis
    state.h = compute_analysis_blue(state.h, B, hist_t['y'][0], R, H)
    hist['ha'][0] = state.h
    
    # run the Nt time steps
    for t in trange(Nt, desc='running BLUE'):
        
        # forecast
        model.forward(state)
        hist['hf'][t+1] = state.h        
        hist['u'][t+1] = state.u
        
        # analysis
        state.h = compute_analysis_blue(state.h, B, hist_t['y'][t+1], R, H)
        hist['ha'][t+1] = state.h
        
    # compute mae and rmse for h
    hist['mae'] = compute_mae(hist_t['h'], hist['ha'])
    hist['rmse'] = compute_rmse(hist_t['h'], hist['ha'])
        
    # return history
    return hist

In [ ]:
# initialise and run the BLUE simulation
state = sw_model.new_state_crenel(h_anom=blue_h_anom)
hist_blue = blue_driver(sw_model, state, hist_truth)

In [ ]:
# make a fancy animation for water height
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h']),
                                   ('BLUE analysis', hist_blue['ha'])],
                                  'Truth vs BLUE simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for horizontal velocity
anim = swm.make_fancy_animation_u([('Truth', hist_truth['u']),
                                   ('BLUE analysis', hist_blue['u'])],
                                  'Truth vs BLUE simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for water height (zoom)
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h']),
                                   ('BLUE analysis', hist_blue['ha']),
                                   ('BLUE forecast', hist_blue['hf'])],
                                  'Truth vs BLUE simulation',
                                  x_min=77,
                                  x_max=83,
                                  y_min=1,
                                  y_max=1.02,
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# plot the time series of MAE
swm.plot_time_series_mae([('Pert. sim.', hist_pert['mae']),
                          ('BLUE analysis', hist_blue['mae'])],
                         title='Error in water height')
plt.show()

In [ ]:
# plot the time series of RMSE
swm.plot_time_series_rmse([('Pert. sim.', hist_pert['rmse']),
                           ('BLUE analysis', hist_blue['rmse'])],
                          title='Error in water height')
plt.show()

## VI. Simulation with ensemble Kalman filter

We then use the stochastic EnKF to assimilate the observations. Compared to the optimal interpolation method used previously, the advantage of the KF is to enable a dynamic representation of the background error covariance matrix $\mathbf{B}$. For large systems however (with more than $10^9$ variables), it is impossible to even store $\mathbf{B}$. This is why the EnKF has been designed to provide a tractable version of the KF. The stochastic EnKF is one possible implementation of the EnKF, which is very simple: during the analysis, the BLUE formula is applied independently to each ensemble member.

For this experiment, we use an ensemble of `Ne=25` members. Each ensemble member is initially a crenel function. The average crenel height is `1` (i.e. no anomaly) et and crenel height standard deviation is `0.02`. Once again, we assume that the observation error covariance matrix $\mathbf{R}$ is the identity matrix.

For this (non-weighted) ensemble-based algorithm, the **analysis** is the (non-weighted) sample mean of the analysis ensemble. It is to be compared to the truth using both the mean absolute error (MAE) and the root mean squared error (RMSE).

### Questions

1. Implement the `compute_covariance` function. Run the assimilation. Comment on the efficiency of the assimilation.
2. Compare the eigenvalues of $\mathbf{B}$ to those of $\mathbf{R}$. What can you conclude? Make the appropriate correction for the assimilation to be effective.
3. Describe the time evolution of the analysis and its error. Explain why the EnKF filter is more efficient than the optimal interpolation.
4. Describe and explain the time evolution of the ensemble spread. Relate this result to the time evolution of the largest eigenvalue of $\mathbf{B}$.
5. Review the `enkf_driver` function. The observations are not perturbed in the analysis. Explain why observations have to be perturbed in the stochastic EnKF, and why it is not an issue in our experiment. 

### Tips

- Use `M.mean(axis=0)` to compute the average of the matrix $\mathbf{M}$ along the first axis.
- Use `M[i]` to select the entire $i$-th row of the matrix $\mathbf{M}$.

In [ ]:
# list of parameters
Ne = 25
mean_h_anom = 1
std_h_anom = 0.02

In [ ]:
# covariance
def compute_covariance(E):
    """Compute the sample covariance matrix of ensemble E.
    
    Arguments
    ---------
    E : numpy array of shape Ne * Nx
        The ensemble matrix.
    
    Returns
    -------
    B : numpy array of shape Nx * Nx
        The covariance matrix of ensemble E.
    """
    # TODO: implement it!
    return np.cov(E, rowvar=False)

# EnKF analysis
def compute_analysis_enkf(Ef, y, R, H):
    """Compute the EnKF analysis."""
    Ne = Ef.shape[0]
    B = compute_covariance(Ef)
    Ea = np.zeros(Ef.shape)
    for i in range(Ne):
        Ea[i] = compute_analysis_blue(Ef[i], B, y, R, H)
    return (Ea, B)

# create a driver
def enkf_driver(model, ensemble, hist_t):
    """Run an EnKF simulation of the given model starting from the given ensemble.
    
    At each time step, an analysis for h is performed using `compute_analysis_enkf`.
    
    Returns the history.
    """
    
    # extract Nt
    Nt = hist_t['Nt']

    # observation error covariance matrix
    R = 1e-3 * np.identity(Ny)

    # observation operator
    H = np.zeros((Ny, model.Nx))
    H[:, 79:82] = np.identity(3)
    
    # allocate memory
    hist = dict(Nt=Nt,
                u=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                hf=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                ha=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                B=np.empty((Nt+1, model.Nx, model.Nx)))
    
    # initialisation
    hist['u'][0] = ensemble.u
    hist['hf'][0] = ensemble.h

    # run first analysis
    ensemble.h, B = compute_analysis_enkf(ensemble.h, hist_t['y'][0], R, H)
    hist['ha'][0] = ensemble.h
    hist['B'][0] = B
    
    # run the Nt time steps
    for t in trange(Nt, desc='running EnKF'):
        
        # forecast
        model.forward_ensemble(ensemble)
        hist['hf'][t+1] = ensemble.h        
        hist['u'][t+1] = ensemble.u
        
        # analysis
        ensemble.h, B = compute_analysis_enkf(ensemble.h, hist_t['y'][t+1], R, H)
        hist['ha'][t+1] = ensemble.h
        hist['B'][t+1] = B
        
    # compute mae and rmse for h
    hist['mae'] = compute_mae(hist_t['h'], hist['ha'].mean(axis=1))
    hist['rmse'] = compute_rmse(hist_t['h'], hist['ha'].mean(axis=1))
        
    # return history
    return hist

In [ ]:
# initialise and run the EnKF simulation
ensemble = sw_model.new_ensemble_crenel(Ne, mean_h_anom, std_h_anom, seed=314)
hist_enkf = enkf_driver(sw_model, ensemble, hist_truth)

In [ ]:
# make a fancy animation for water height
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h']),
                                   ('EnKF analysis', hist_enkf['ha'].mean(axis=1))],
                                  'Truth vs EnKF simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for horizontal velocity
anim = swm.make_fancy_animation_u([('Truth', hist_truth['u']),
                                   ('EnKF analysis', hist_enkf['u'].mean(axis=1))],
                                  'Truth vs EnKF simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# compute the largest eigenvalue of B
largest_eigval = np.zeros(Nt+1)
for t in range(Nt+1):
    largest_eigval[t] = abs(scipy.linalg.eigvals(hist_enkf['B'][t])).max()

In [ ]:
# plot the time evolution of the largest eigenvalue
fig = plt.figure(figsize=(12, 6))
ax = plt.gca()
ax.set_yscale('log')
ax.set_xlim(0, 15)
ax.set_ylim(1e-5, 1)
ax.set_xlabel('Time')
ax.set_ylabel('Module of largest B eigenvalue')
time = sw_model.dt * np.arange(Nt+1)
ax.plot(time, largest_eigval, c=palette[0])
plt.show()

In [ ]:
# plot the time series of MAE
swm.plot_time_series_mae([('Pert. sim.', hist_pert['mae']),
                          ('BLUE analysis', hist_blue['mae']),
                          ('EnKF analysis', hist_enkf['mae'])],
                         title='Error in water height')
plt.show()

In [ ]:
# plot the time series of RMSE
swm.plot_time_series_rmse([('Pert. sim.', hist_pert['rmse']),
                          ('BLUE analysis', hist_blue['rmse']),
                          ('EnKF analysis', hist_enkf['rmse'])],
                         title='Error in water height')
plt.show()

In [ ]:
# make a fancy animation for B
anim = swm.make_fancy_animation_B(hist_enkf['B'], 'B matrix (EnKF)')
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for water height (with ensemble)
anim = swm.make_fancy_animation_h_ensemble([('Truth', hist_truth['h']),
                                            ('EnKF analysis', hist_enkf['ha'].mean(axis=1))],
                                           [hist_enkf['ha'][:, i] for i in range(Ne)],
                                           'Truth vs EnKF simulation',
                                           freq=10)
HTML(anim.to_jshtml())

## VII. Co-assimilation of both h and u

In the previous experiments, the BLUE formula has been applied to $h(x)$. In order to apply the BLUE formula to both $h(x)$ and $u(x)$, we need to specify the background error covariance matrix $\mathbf{B}$ for the entire state (_i.e._, the `2Nx` variables). When using the EnKF, the covariances can all be extracted from the ensemble. This is what is done here.

### Questions

1. In the previous experiments, $u(x)$ was not corrected during the analysis, but it got corrected anyway. Where did the correction come from?
2. Read the updated `co_enkf_driver` function and describe the modifications (compared to the original `enkf_driver` function).
3. Run the simulation and explain why the $(h, u)$ co-assimilation is more efficient than the single $h$ assimilation.

In [ ]:
# create a driver
def co_enkf_driver(model, ensemble, hist_t):
    """Run an EnKF simulation of the given model starting from the given ensemble.
    
    At each time step, an analysis for h is performed using `compute_analysis_enkf`.
    
    Returns the history.
    """
    
    # extract Nt
    Nt = hist_t['Nt']

    # observation error covariance matrix
    R = 1e-4 * np.identity(Ny)

    # observation operator
    H = np.zeros((Ny, 2*model.Nx))
    H[:, 79:82] = np.identity(3)
    
    # allocate memory
    hist = dict(Nt=Nt,
                uf=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                ua=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                hf=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                ha=np.empty((Nt+1, ensemble.Ne, model.Nx)),
                B=np.empty((Nt+1, 2*model.Nx, 2*model.Nx)))
    
    # initialisation
    hist['uf'][0] = ensemble.u
    hist['hf'][0] = ensemble.h

    # run first analysis
    Ef = np.concatenate([ensemble.h, ensemble.u], axis=1)
    Ea, B = compute_analysis_enkf(Ef, hist_t['y'][0], R, H)
    hist['ha'][0] = Ea[:, :model.Nx]
    hist['ua'][0] = Ea[:, model.Nx:]
    hist['B'][0] = B
    ensemble.h[:] = Ea[:, :model.Nx]
    ensemble.u[:] = Ea[:, model.Nx:]
    
    # run the Nt time steps
    for t in trange(Nt, desc='running EnKF'):
        
        # forecast
        model.forward_ensemble(ensemble)
        hist['hf'][t+1] = ensemble.h        
        hist['uf'][t+1] = ensemble.u
        
        # analysis
        Ef = np.concatenate([ensemble.h, ensemble.u], axis=1)
        Ea, B = compute_analysis_enkf(Ef, hist_t['y'][t+1], R, H)
        hist['ha'][t+1] = Ea[:, :model.Nx]
        hist['ua'][t+1] = Ea[:, model.Nx:]
        hist['B'][t+1] = B
        ensemble.h[:] = Ea[:, :model.Nx]
        ensemble.u[:] = Ea[:, model.Nx:]
        
    # compute mae and rmse for h
    hist['mae'] = compute_mae(hist_t['h'], hist['ha'].mean(axis=1))
    hist['rmse'] = compute_rmse(hist_t['h'], hist['ha'].mean(axis=1))
        
    # return history
    return hist

In [ ]:
# initialise and run the EnKF simulation
ensemble = sw_model.new_ensemble_crenel(Ne, mean_h_anom, std_h_anom, seed=42)
hist_co_enkf = co_enkf_driver(sw_model, ensemble, hist_truth)

In [ ]:
# make a fancy animation for water height
anim = swm.make_fancy_animation_h([('Truth', hist_truth['h']),
                                   ('EnKF analysis', hist_enkf['ha'].mean(axis=1)),
                                   ('(co-)EnKF analysis', hist_co_enkf['ha'].mean(axis=1))],
                                  'Truth vs EnKF simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for horizontal velocity
anim = swm.make_fancy_animation_u([('Truth', hist_truth['u']),
                                   ('EnKF analysis', hist_enkf['u'].mean(axis=1)),
                                   ('(co-)EnKF analysis', hist_co_enkf['ua'].mean(axis=1))],
                                  'Truth vs EnKF simulation',
                                  freq=10)
HTML(anim.to_jshtml())

In [ ]:
# make a fancy animation for B
anim = swm.make_fancy_animation_B(hist_co_enkf['B'], 'B matrix (EnKF, co-assimilation)')
HTML(anim.to_jshtml())

In [ ]:
# plot the time series of MAE
swm.plot_time_series_mae([('Pert. sim.', hist_pert['mae']),
                          ('BLUE analysis', hist_blue['mae']),
                          ('EnKF analysis', hist_enkf['mae']),
                          ('(co-)EnKF analysis', hist_co_enkf['mae'])],
                         title='Error in water height')
plt.show()

In [ ]:
# plot the time series of RMSE
swm.plot_time_series_rmse([('Pert. sim.', hist_pert['rmse']),
                          ('BLUE analysis', hist_blue['rmse']),
                          ('EnKF analysis', hist_enkf['rmse']),
                          ('(co-)EnKF analysis', hist_co_enkf['rmse'])],
                         title='Error in water height')
plt.show()